In [918]:
# read in data
# add season info and holiday info


import pandas as pd
import numpy as np
import pyro
import torch

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
from pyro.nn import PyroModule, PyroSample


from sklearn.metrics import mean_squared_error

In [919]:
%matplotlib inline
import matplotlib.pyplot as plt

pyro.set_rng_seed(30)

In [920]:
data = pd.read_csv('data/data_clean.csv', parse_dates=['Week'])

In [921]:
data

,Week,Display,EndCap,PriceChange,TV,Facebook,Twitter,Amazon,Audio,Print,Digital_AO,sales
0,2017-01-01,27.63,54.41,0,18918856,15572499,256513,789289,0,0,6715999,430447.6
1,2017-01-08,28.84,54.00,0,0,19619339,154474,3365996,0,0,1718752,431072.0
2,2017-01-15,31.67,58.35,0,0,0,229481,0,0,0,248176,375697.6
3,2017-01-22,32.53,57.90,0,26947686,11375713,160959,0,0,0,508693,429284.1
4,2017-01-29,28.14,53.30,0,0,10840061,0,0,0,0,0,400453.1
...,...,...,...,...,...,...,...,...,...,...,...,...
143,2019-09-29,31.51,53.76,1,0,0,131028,0,0,0,116116,345999.4
144,2019-10-06,27.45,54.00,1,0,0,0,299885,0,0,0,352412.2
145,2019-10-13,26.46,52.90,1,0,0,0,0,0,0,0,355926.3
146,2019-10-20,29.43,57.79,1,21600651,0,190354,0,0,0,152207,395565.0


In [922]:
# add season information
data['is_spring'] = np.where(data['Week'].dt.month.isin([3, 4, 5]), 1, 0)
data['is_summer'] = np.where(data['Week'].dt.month.isin([6, 7, 8]), 1, 0)
data['is_fall'] = np.where(data['Week'].dt.month.isin([9, 10, 11]), 1, 0)
data['is_winter'] = np.where(data['Week'].dt.month.isin([12, 1, 2]), 1, 0)

# add holiday information
# define holiday weeks as the 47th week(Thanksgiving), last week of the year (Christmas), the first week of year (New Year)
data['is_holiday'] = np.where(data['Week'].dt.isocalendar().week.isin([47, 52, 1]), 1, 0)


In [923]:
data

,Week,Display,EndCap,PriceChange,TV,Facebook,Twitter,Amazon,Audio,Print,Digital_AO,sales,is_spring,is_summer,is_fall,is_winter,is_holiday
0,2017-01-01,27.63,54.41,0,18918856,15572499,256513,789289,0,0,6715999,430447.6,0,0,0,1,1
1,2017-01-08,28.84,54.00,0,0,19619339,154474,3365996,0,0,1718752,431072.0,0,0,0,1,1
2,2017-01-15,31.67,58.35,0,0,0,229481,0,0,0,248176,375697.6,0,0,0,1,0
3,2017-01-22,32.53,57.90,0,26947686,11375713,160959,0,0,0,508693,429284.1,0,0,0,1,0
4,2017-01-29,28.14,53.30,0,0,10840061,0,0,0,0,0,400453.1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2019-09-29,31.51,53.76,1,0,0,131028,0,0,0,116116,345999.4,0,0,1,0,0
144,2019-10-06,27.45,54.00,1,0,0,0,299885,0,0,0,352412.2,0,0,1,0,0
145,2019-10-13,26.46,52.90,1,0,0,0,0,0,0,0,355926.3,0,0,1,0,0
146,2019-10-20,29.43,57.79,1,21600651,0,190354,0,0,0,152207,395565.0,0,0,1,0,0


In [924]:
# extend the media data to contain previous results
media_df = data[['TV', 'Facebook', 'Twitter', 'Amazon', 'Audio', 'Print', 'Digital_AO']]


In [925]:
tensor_list = []
media_row_count = media_df.shape[0]
media_col_count = media_df.shape[1]

max_lag = 9
for i in range(max_lag):
    current_media_data = torch.tensor(media_df.values[i:(media_row_count-max_lag+i+1)])
    current_media_data = torch.reshape(current_media_data, (media_row_count - max_lag + 1, media_col_count, 1))
    tensor_list.insert(0, current_media_data)

media_tensor = torch.cat(tensor_list, dim=2)

In [926]:
sale_tensor = torch.tensor(data['sales'].values[(max_lag-1):]) / 1000

In [927]:
base_tensor = torch.tensor(data[['is_spring', 'is_summer', 'is_fall', 'is_winter', 'is_holiday']].values[(max_lag-1):])

In [928]:
trade_tensor = torch.tensor(data[['Display', 'EndCap']].values[(max_lag-1):])

In [929]:
price_tensor = torch.tensor(data[['PriceChange']].values[(max_lag-1):])

In [930]:
#I used Bayesian Hierarchical modeling to construct the Marketing Mix Model.
#1. For the media data, adstock transformation was applied to media data to model the lagging effects.
#2. Sale was constructed by adding together the contributions of media, trade, base and noise.
def adstock_model(media_data, trade_data, price_data, base_data, sale_data, max_lag=8):
    media_col_count = media_data.shape[1]
    trade_col_count = trade_data.shape[1]
    base_col_count = base_data.shape[1]
    
    lag = torch.tensor(np.arange(0, max_lag))
    lag = lag.repeat(media_col_count, 1)
    retain_rate = pyro.sample('retain_rate', dist.Uniform(0, 1))
    delay = pyro.sample('delay', dist.Uniform(0,  max_lag/2))
    
    # adstock parameters
    weight = torch.pow(retain_rate, (lag - delay)**2)
    weight = weight/torch.sum(weight)
    
    #weight = torch.reshape(weight, (media_col_count, 1))
    beta_ads = pyro.sample('beta_ads', dist.HalfNormal(10 * torch.ones(media_col_count)).to_event(1))
    
    beta_price = pyro.sample('beta_price', dist.Normal(0, 10))
    beta_trade = pyro.sample('beta_trade', dist.HalfNormal(10 * torch.ones(trade_col_count)).to_event(1))
    beta_base = pyro.sample('beta_base', dist.Normal(torch.zeros(base_col_count), 10 * torch.ones(base_col_count)).to_event(1))
    
    intercept = pyro.sample('intercept', dist.Uniform(0, torch.min(sale_data)))
    
    noise = pyro.sample('noise', dist.Normal(0, 10))

    with pyro.plate('data', len(media_data)):
        adstock_mean = torch.sum(weight * media_data, dim=2)
        
        y_expected = torch.sum(adstock_mean * beta_ads, dim=1) + torch.sum(beta_trade * trade_data, dim=1) + torch.sum(beta_base *base_data, dim=1) + torch.sum(beta_price * price_data, dim=1) + intercept
        
        pyro.sample('y', dist.Normal(y_expected, 1), obs=sale_data)
        

In [931]:
def compute_prediction(hmc_samples, media_data, trade_data, price_data, base_data):
    retain_rate = hmc_samples['retain_rate'].mean()
    delay = hmc_samples['delay'].mean()
    
    beta_ads = torch.tensor(hmc_samples['beta_ads'].mean(0))
    beta_trade = torch.tensor(hmc_samples['beta_trade'].mean(0))
    beta_base = torch.tensor(hmc_samples['beta_base'].mean(0))
    
    beta_price = hmc_samples['beta_price'].mean()
    
    lag = torch.tensor(np.arange(0, max_lag))
    lag = lag.repeat(media_col_count, 1)
    weight = torch.pow(retain_rate, (lag - delay)**2)
    weight = weight/torch.sum(weight)
    
    adstock = torch.sum(weight * media_data, dim=2)
    
    ads_prediction = torch.sum(adstock * beta_ads, dim=1) 
    trade_prediction = torch.sum(beta_trade * trade_data, dim=1) + torch.sum(beta_price * price_data, dim=1)
    base_prediction = torch.sum(beta_base * base_data, dim=1) + hmc_samples['intercept'].mean()
    
    y_prediction = ads_prediction + trade_prediction + base_prediction
    return (y_prediction.numpy(), ads_prediction.numpy(), trade_prediction.numpy(), base_prediction.numpy())
    

In [932]:
hmc_samples_list = []
predictions_list = []
errors = []

for i in range(5):
    kernel_bayes= NUTS(adstock_model)
    mcmc_bayes = MCMC(kernel_bayes, num_samples=1000, warmup_steps=500)
    mcmc_bayes.run(media_tensor, trade_tensor, price_tensor, base_tensor, sale_tensor, max_lag=max_lag)


    hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc_bayes.get_samples().items()}
    predictions = compute_prediction(hmc_samples, media_tensor, trade_tensor, price_tensor, base_tensor)
    
    hmc_samples_list.append(hmc_samples)
    predictions_list.append(predictions_list)
    
    errors.append(mean_squared_error(predictions[0] * 1000, data['sales'].values[(max_lag-1):]))



Warmup:  14%|█▍        | 216/1500 [04:02,  1.34s/it, step size=1.96e-02, acc. prob=0.809]

KeyboardInterrupt: 

In [913]:
# The accuracy of the model is accessed by mean squared error between the actual sale and predicted sale. Low mean squared errors suggest that the model is working as expected. 

print("Minimun mean squared error: " + str(min(errors)))

min_index = errors.index(min(errors))

hmc_samples = hmc_samples_list[min_index]
predictions = compute_prediction(hmc_samples, media_tensor, trade_tensor, price_tensor, base_tensor)


Minimun mean squared error: 88204771.53510393


In [905]:
print("retain_rate")
print(hmc_samples['retain_rate'].mean())

array([0.47949744, 0.18947633])

In [906]:
print("delay")
print(hmc_samples['delay'].mean())

-0.7927336457474662

In [914]:
print("\nQuestion 1\n")
mean_ads = predictions[1].mean()
mean_trade = predictions[2].mean()
mean_base = predictions[3].mean()
mean_total = predictions[0].mean()

print("Relative contribution from base: " + str(mean_base/mean_total))
print("Relative contribution from trade: " + str(mean_trade/mean_total))
print("Relative contribution from media: " + str(mean_ads/mean_total))


Question 1

Relative contribution from base: 0.821362615057443
Relative contribution from trade: 0.06079506153140223
Relative contribution from media: 0.11784232341115465


In [915]:
#'TV', 'Facebook', 'Twitter', 'Amazon', 'Audio', 'Print', 'Digital_AO'
print("\nQuestion 2\n")
media_contributions = hmc_samples['beta_ads'].mean(0) * 1.28 * 10e9
print("Sales (USD) contributed per GRP: ")
print("TV: " + str(media_contributions[0]))
print("Facebook: " + str(media_contributions[1]))
print("Twitter: " + str(media_contributions[2]))
print("Amazon: " + str(media_contributions[3]))
print("Audio: " + str(media_contributions[4]))
print("Print: " + str(media_contributions[5]))
print("Digital_AO: " + str(media_contributions[6]))

print("\nThe most effective media is TV")
print("The least effective media is Audio")


Question 2

Sales (USD) contributed per GRP: 
TV: 282867.4671841145
Facebook: 239255.51274085414
Twitter: 12171.271117610208
Amazon: 32284.42430505261
Audio: 3698.7407093173865
Print: 80732.52910988693
Digital_AO: 38007.08605147453

The most effective media is Twitter
The least effective media is Amazon


In [916]:
print("\nQuestion 4\n")

trade_contributions = hmc_samples['beta_trade'].mean(0) * 1000

print("Effect of trade activity measured by Sales (USD) contributed per 1% trade activity: ")
print("Display: " + str(trade_contributions[0]))
print("EndCap: " + str(trade_contributions[1]))


Question 4

Effect of trade activity as measured by Sales (USD) contributed per 1% trade activity: 
Display: 479.4974440248601
EndCap: 189.47633315518982


In [917]:
print("\nQuestion 5\n")

price_contribution = hmc_samples['beta_price'].mean() * 1000
print("Effect of price change on Sales: " + str(price_contribution))


Question 5

Effect of price change on Sales: -792.7336457474662
